<a href="https://colab.research.google.com/github/rizzken/PreAppointmentPatientSummary-Eval/blob/main/02_summary_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Choose sample data for the further ML testing
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/patients_with_age.csv')

samples = df.sample(10)
print (samples)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                                       id   birthdate   deathdate  \
33   02bdcf90-32f9-f93f-79d8-5a1933f2af06  1992-09-13  1958-05-21   
107  f70a6900-4871-11c4-cbb2-4ee86975558c  1960-01-06         NaN   
90   17d6e339-f882-e88c-320e-c3f71c62b945  1993-06-27         NaN   
3    43040569-f355-dcc7-1fa0-c2d386a57cc7  2007-02-18         NaN   
14   d74d24e5-192e-03f7-5bda-6cd8195bbefe  2011-09-16         NaN   
100  fb1ff537-0e7d-77ed-b2be-3cbd195cc64b  2004-03-24         NaN   
73   d70a6bbf-8ebd-77e0-2f04-d3369c9f0fcb  1963-04-21         NaN   
22   db4ce617-081b-c16a-270f-8f8391bee5d3  2006-09-25         NaN   
74   b2ec1b43-0b3b-eec8-e2ec-21d19a1033d6  2002-01-06         NaN   
28   c5a11ac2-b86c-7f70-de60-5daed3edc3d9  1953-03-05         NaN   

             ssn     drivers    passport prefix        first           last  \
33   999-29-6086  99938340.0  X8

###Patient ID: 02bdcf90-32f9-f93f-79d8-5a1933f2af06
<details>
<summary>Original Record (click to expand)</summary>

**Full record:**  
- Name: Mr. Richie600 McCullough561
- Race: white,nonhispanic
- Gender: male  
- Birthdate: 1992-09-13  
- Deathdate: 1958-05-21 (should be replaced with NaN)  
- Age: (not calculated due to invalid deathdate)  
- SSN: 999-29-6086  
- Passport: X84875389X  
- Drivers: 99938340  
- Birthplace Daphne, Alabama, US  
- Address: 912 Goyette Ranch  
- City: Center Point  
- State: Alabama  
- County: Jefferson county  
- Fips: 1073  
- Zip: 35215  
- Healthcare coverage: 14104.06  
- Income: 30043.0  
- Location: POINT(-86.60579960825079 33.668945899945584)

</details>

**Manual Summary (prepared by QA)**

**Patient**: Mr. Richie600 McCullough561, adult male, white  
**Age**: Not calculated (invalid deathdate)  
**Marital status**: Unknown (missing)  
**Location**: Center Point, Alabama, US
**Income**: 30,043 per year  
**Healthcare coverage**: 14,104.06  
**Healthcare expenses**: Unknown (missing)  
**Key notes**:  
- Invalid deathdate (earlier than birthdate) — data error  
- No medical history, allergies or procedures in demographics  
- SSN, Passport and Drivers present — full identification available

**Quick Evaluation**  
- Coverage: partial  
- Hallucinations: none  
- Factual errors: yes  
- Score: 3/5


## General Evaluation Notes & Observations (after 10 samples)

### Key patterns and recurring issues

- **Invalid dates** — в 4 из 10 записей birthdate или deathdate некорректны (будущие даты рождения или deathdate раньше birthdate). Это приводит к невозможности расчёта возраста и ломает summary.  
- **Missing critical demographics** — marital status отсутствует в 6 из 10 случаев, race/ethnicity — в 3–4.  
- **Geographical mismatch** — все координаты и адреса указывают на США (Alabama, etc.), а не на NZ. Модель может генерировать нерелевантный контекст.  
- **Negative financial values** — в 2–3 записях отрицательный income или expenses — это критическая ошибка данных.  
- **Hallucinations** — в большинстве случаев отсутствуют (модель не придумывает лишнего), но иногда может добавить несуществующие поля (например, occupation).  
- **Coverage of key fields** — средний score 2.8/5. Лучше всего покрываются пол, имя, доход. Хуже всего — возраст и marital status.

### Recommendations for improvement

1. **Data cleaning** — исправить отрицательные значения, будущие даты, стандартизировать gender и заполнить пропуски в marital/race.  
2. **Prompt engineering** — явно указывать в промпте: "Use only provided data. Do not invent occupation or medical history."  
3. **Location adjustment** — добавить NZ-контекст (заменить координаты/города) для реалистичности.  
4. **Model choice** — протестировать более современные модели (Llama-3.1, Mistral) или добавить RAG для проверки фактов.  
5. **Evaluation** — в следующих итерациях добавить авто-метрики (ROUGE, faithfulness) и сравнить zero-shot vs few-shot.

Средний score по 10 примерам: 2.8/5  
Основная слабость — некорректные данные (даты, финансы), а не сама модель.